In [1]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm import tqdm
import pandas as pd
import torch
from transformers import GPT2LMHeadModel

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device("cuda:0")

In [2]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [3]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
Chatbot_Data = pd.read_csv("../../Data_preprocessing/custom_chatbotdataset(Training).csv")
Chatbot_Data_validation = pd.read_csv("../../Data_preprocessing/custom_chatbotdataset(Validation).csv")

In [ ]:
Chatbot_Data.shape

(145954, 3)

In [ ]:
Chatbot_Data.head(20)

,label,Q,A
0,9,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,9,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,9,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,9,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,9,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
5,9,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,진로에 대해서 고민하고 계시는군요. 어떤 점이 고민인가요?
6,9,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,부모님의 노여움에 섭섭하시군요. 이런 상황을 어떻게 해결하면 좋을까요?
7,11,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?
8,2,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,취업에 대해 걱정이 되는군요.
9,11,요즘 직장생활이 너무 편하고 좋은 것 같아!,직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?


In [ ]:
# # 주어진 감정 딕셔너리
# emotion_map = {
#     '노여워하는': 9, '느긋': 11, '걱정스러운': 2, '당혹스러운': 12, '당황': 13, '마비된': 15,
#     '만족스러운': 16, '배신당한': 18, '버려진': 19, '부끄러운': 20, '분노': 21, '불안': 22,
#     '비통한': 23, '상처': 24, '성가신': 25, '스트레스 받는': 26, '슬픔': 27, '신뢰하는': 28,
#     '신이 난': 29, '실망한': 30, '악의적인': 31, '안달하는': 32, '안도': 33, '억울한': 34,
#     '열등감': 35, '염세적인': 36, '외로운': 37, '우울한': 38, '고립된': 3, '좌절한': 41,
#     '후회되는': 55, '혐오스러운': 51, '한심한': 50, '자신하는': 39, '기쁨': 6, '툴툴대는': 48,
#     '남의 시선을 의식하는': 8, '회의적인': 54, '죄책감의': 42, '혼란스러운': 52, '초조한': 45,
#     '흥분': 56, '충격 받은': 46, '취약한': 47, '편안한': 49, '방어적인': 17, '질투하는': 43,
#     '두려운': 14, '눈물이 나는': 10, '짜증내는': 44, '조심스러운': 40, '낙담한': 7,
#     '환멸을 느끼는': 53, '희생된': 57, '감사하는': 1, '구역질 나는': 5, '괴로워하는': 4,
#     '가난한, 불우한': 0
# }

emotion2Token_map = {
    9: '<unused10>', 11: '<unused11>', 2: '<unused12>', 12: '<unused13>', 13: '<unused14>', 15: '<unused15>',
    16: '<unused16>', 18: '<unused17>', 19: '<unused18>', 20: '<unused19>', 21: '<unused20>', 22: '<unused21>',
    23: '<unused22>', 24: '<unused23>', 25: '<unused24>', 26: '<unused25>', 27: '<unused26>', 28: '<unused27>',
    29: '<unused28>', 30: '<unused29>', 31: '<unused30>', 32: '<unused31>', 33: '<unused32>', 34: '<unused33>',
    35: '<unused34>', 36: '<unused35>', 37: '<unused36>', 38: '<unused37>', 3: '<unused38>', 41: '<unused39>',
    55: '<unused40>', 51: '<unused41>', 50: '<unused42>', 39: '<unused43>', 6: '<unused44>', 48: '<unused45>',
    8: '<unused46>', 54: '<unused47>', 42: '<unused48>', 52: '<unused49>', 45: '<unused50>', 56: '<unused51>',
    46: '<unused52>', 47: '<unused53>', 49: '<unused54>', 17: '<unused55>', 43: '<unused56>', 14: '<unused57>',
    10: '<unused58>', 44: '<unused59>', 40: '<unused60>', 7: '<unused61>', 53: '<unused62>', 57: '<unused63>',
    1: '<unused64>', 5: '<unused65>', 4: '<unused66>', 0: '<unused67>'
}

In [ ]:
for i in range(58):
    Chatbot_Data.replace({'label' :i}, emotion2Token_map[i], inplace = True)

In [ ]:
Chatbot_Data.head(20)

,label,Q,A
0,<unused10>,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,<unused10>,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,<unused10>,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,<unused10>,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,<unused10>,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
5,<unused10>,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,진로에 대해서 고민하고 계시는군요. 어떤 점이 고민인가요?
6,<unused10>,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,부모님의 노여움에 섭섭하시군요. 이런 상황을 어떻게 해결하면 좋을까요?
7,<unused11>,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?
8,<unused12>,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,취업에 대해 걱정이 되는군요.
9,<unused11>,요즘 직장생활이 너무 편하고 좋은 것 같아!,직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?


In [ ]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        
        emotion = turn["label"]
        
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(emotion + self.sent_token + self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (np.array(token_ids), np.array(mask), np.array(labels_ids))

In [ ]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
train_dataloader = DataLoader(train_set, batch_size=128, num_workers=4, shuffle=False)

valid_set = ChatbotDataset(Chatbot_Data_validation, max_len=40)
valid_dataloader = DataLoader(valid_set, batch_size=128, num_workers=4, shuffle=False)

In [ ]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 120
Sneg = -1e18

In [ ]:
print ("start")
for epoch in range(epoch):
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    for samples in tqdm(train_dataloader): # train
        optimizer.zero_grad()
        token_ids, mask, label = samples
        
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)
        
        out = model(token_ids)
        
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
        train_loss += avg_loss
    train_loss = ((train_loss) / len(train_dataloader))
    print('Epoch = {}, train loss = {}'.format((epoch+1), train_loss))
        
    with torch.no_grad(): # validation
        model.eval()
        for samples in tqdm(valid_dataloader):
            token_ids, mask, label = samples
            
            token_ids = token_ids.to(device)
            mask = mask.to(device)
            label = label.to(device)
            
            out = model(token_ids)
            
            out = out.logits
            mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
            mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
            loss = criterion(mask_out.transpose(2, 1), label)
            
            avg_loss = loss.sum() / mask.sum()

            valid_loss += avg_loss
        valid_loss = ((valid_loss) / len(valid_dataloader))
        print('Epoch = {}, validation loss = {}'.format((epoch+1), valid_loss))
            
print ("end")

start


  0%|          | 0/1141 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
100%|██████████| 1141/1141 [03:11<00:00,  5.94it/s]


Epoch = 1, train loss = 20.41384506225586


100%|██████████| 141/141 [00:09<00:00, 15.55it/s]


...

100%|██████████| 1141/1141 [03:12<00:00,  5.94it/s]


Epoch = 5, train loss = 19.947525024414062


100%|██████████| 141/141 [00:09<00:00, 15.46it/s]


Epoch = 5, validation loss = 19.728927612304688


 87%|████████▋ | 990/1141 [02:46<00:25,  5.95it/s]

In [ ]:
torch.save({
                'epoch': epoch+1,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, 'model_EQ2A_Custom_Data_label2map_{}_.pth'.format(epoch+1))

## 챗봇 테스트

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + '0' + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids.to(device))
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))